In [1]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())


import time
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
executed in 1.00 seconds.


In [2]:
# import the tidy3d package and configure it with your API key
import numpy as np
import tidy3d as td
import tidy3d.web as web
# web.configure("YOUR API KEY") # if authentication needed, uncomment this line and paste your API key here

# set up global parameters of simulation ( speed of light / wavelength in micron )
freq0 = td.C_0 / 0.75

# create structure - a box centered at 0, 0, 0 with a size of 1.5 micron and permittivity of 2
square = td.Structure(
    geometry=td.Box(center=(0, 0, 0), size=(1.5, 1.5, 1.5)),
    medium=td.Medium(permittivity=2.0)
)

# create source - A point dipole source with frequency freq0 on the left side of the domain
source = td.PointDipole(
    center=(-1.5, 0, 0),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 10.0),
    polarization="Ey",
)

# create monitor - Measures electromagnetic fields within the entire domain at z=0
monitor = td.FieldMonitor(
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="fields",
    colocate=True,
)

Let's make a list of simulations with different variations:

In [3]:
simulations = list()
for wavelength_step_i in [15, 20, 25]:
    # Initialize simulation - Combine all objects together into a single specification to run
    sim = td.Simulation(
        size=(4, 3, 3),
        grid_spec=td.GridSpec.auto(min_steps_per_wvl=wavelength_step_i),
        structures=[square],
        sources=[source],
        monitors=[monitor],
        run_time=120/freq0,
    )
    simulations.append(sim)

print(f"simulation grid is shaped {sim.grid.num_cells} for {int(np.prod(sim.grid.num_cells)/1e6)} million cells.")

simulation grid is shaped [179, 147, 147] for 3 million cells.


## Deploying Simulations Synchronously

If we wanted to run things synchronously, we can just iterate over a `web.run` command:

We can also create a batch:

In [4]:
# TODO

## Deploying Simulations Asynchronously

Now, let's explore multiple ways of running this simulation asynchronously:

In [5]:
from tidy3d.web.api.asynchronous import run

In [6]:
[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations]

[<coroutine object run at 0x77efff698660>,
 <coroutine object run at 0x77efff6985f0>,
 <coroutine object run at 0x77efff6986d0>]

In [7]:
async def main():
    await asyncio.gather(*[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations])

await main()

13:03:01 CEST Created task 'quickstart' with task_id                            
              'fdve-319ad3ab-26e4-44ad-852d-ea42121373ca' and task_type 'FDTD'.

              View task using web UI at                                         
              ]8;id=268685;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=660728;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\taskId]8;;\]8;id=268685;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\=]8;;\]8;id=408810;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\fdve]8;;\]8;id=268685;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\-319ad3ab-26]8;;\
              ]8;id=268685;https://tidy3d.simulation.cloud/workbench?taskId=fdve-319ad3ab-26e4-44ad-852d-ea42121373ca\e4-44ad-852d-ea42121373ca']8;;\.

Output()

13:03:04 CEST status = queued

              To cancel the simulation, use 'web.abort(task_id)' or             
              'web.delete(task_id)' or abort/delete the task in the web UI.     
              Terminating the Python script will not stop the job running on the
              cloud.

...

Output()

13:04:12 CEST status = success

              View simulation result at                                         
              ]8;id=560461;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=980209;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\taskId]8;;\]8;id=560461;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\=]8;;\]8;id=199829;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\fdve]8;;\]8;id=560461;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\-57bb6b5c-b1]8;;\
              ]8;id=560461;https://tidy3d.simulation.cloud/workbench?taskId=fdve-57bb6b5c-b1ea-4ca1-a566-2dba595695e8\ea-4ca1-a566-2dba595695e8']8;;\.

Output()

13:04:15 CEST loading simulation from data/data.hdf5